In [1]:
from database.market import Market
from tqdm import tqdm
from datetime import datetime, timezone, timedelta
import pandas_datareader as pdr
from extractor.finnhub_extractor import FinnhubExtractor
from extractor.tiingo_extractor import TiingoExtractor
from time import sleep
from database.strategy import Strategy
import pandas as pd


In [2]:
start = (datetime.now() - timedelta(days=2555)).strftime("%Y-%m-%d")
end = datetime.now().strftime("%Y-%m-%d")
market = Market()
strat_db = Strategy("portfolio")
datasets = ["pdr"
#             ,"tiingo"
#             ,"finnhub"
           ]

In [3]:
# market.connect()
# market.drop_table("sp500")
# df = pd.read_html("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies",attrs={"id":"constituents"},header=0)[0]
# market.store_data("sp500",df)
# market.close()

In [4]:
market.connect()
tickers = market.retrieve_data("sp500").sort_values("Symbol")["Symbol"]
market.close()

In [5]:
strat_db.connect()
strat_db.drop_table("pdr_prices")
# strat_db.drop_table("tiingo_prices")
# strat_db.drop_table("finnhub_prices")
fails = []
for ticker in tqdm(tickers):
    try:
        if "." in ticker:
            ticker = ticker.replace(".","-")
        status = "tiingo"
        price = pdr.DataReader(ticker, 
                       start=start, 
                       end=end, 
                       data_source='yahoo')
        price["ticker"] = ticker
        strat_db.store_data("pdr_prices",price.reset_index())
#         tp = TiingoExtractor(ticker,start,end).extract()
#         tp["ticker"] = ticker
#         strat_db.store_data("tiingo_prices",tp.reset_index())
#         fp = FinnhubExtractor.extract(ticker,start,end)
#         fp = pd.DataFrame(fp)
#         fp["ticker"] = ticker
#         strat_db.store_data("finnhub_prices",fp.reset_index())
    except Exception as e:
        msg = {"status":status,"ticker":ticker,"exception":str(e)}
        print(msg,ticker)
        fails.append(ticker)
strat_db.close()

 25%|███████████████████████████████████████████▌                                                                                                                                      | 124/506 [03:02<09:19,  1.46s/it]

{'status': 'tiingo', 'ticker': 'CXO', 'exception': 'No data fetched for symbol CXO using YahooDailyReader'} CXO


 86%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍                        | 436/506 [12:46<01:47,  1.53s/it]

{'status': 'tiingo', 'ticker': 'TIF', 'exception': 'No data fetched for symbol TIF using YahooDailyReader'} TIF


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 506/506 [14:56<00:00,  1.77s/it]


In [6]:
price

,High,Low,Open,Close,Volume,Adj Close,ticker
Date,,,,,,,
2014-04-21,29.059999,28.530001,28.700001,29.030001,4437500.0,27.578152,ZTS
2014-04-22,30.020000,29.190001,29.520000,29.889999,8353800.0,28.395145,ZTS
2014-04-23,29.889999,29.299999,29.750000,29.389999,3803300.0,27.987564,ZTS
2014-04-24,30.020000,29.250000,29.299999,29.889999,4412100.0,28.533607,ZTS
2014-04-25,30.290001,29.620001,29.809999,30.160000,4857700.0,28.791353,ZTS
...,...,...,...,...,...,...,...
2021-04-12,163.149994,158.520004,160.070007,162.509995,2916400.0,162.509995,ZTS
2021-04-13,164.000000,161.490005,162.199997,163.470001,2452700.0,163.470001,ZTS
2021-04-14,164.839996,162.630005,164.009995,163.080002,1183100.0,163.080002,ZTS
